<a href="https://colab.research.google.com/github/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/DeepLearning/PLN/Torah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!kill -9 -1 #reiniciando la maquina virtual

In [ ]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

In [ ]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


In [ ]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)   

In [ ]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.6845605359999922
GPU (s):
0.053426725000008446
GPU speedup over CPU: 68x


In [ ]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/device:GPU:0') #activando la GPU 

In [ ]:
fileDL= tf.keras.utils.get_file('Torah.txt','https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Cuentos_txt/Torah.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')


In [ ]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:4906052 caracteres
el texto esta compuesto de estos :37 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'las escrituras de restauracion edicion del nombre verdadero con un contenido tanto de la tanaj y el '
array([22, 11, 29,  0, 15, 29, 13, 28, 19, 30, 31, 28, 11, 29,  0, 14, 15,
        0, 28, 15, 29, 30, 11, 31, 28, 11, 13, 19, 25, 24,  0, 15, 14, 19,
       13, 19, 25, 24,  0, 14, 15, 22,  0, 24, 25, 23, 12, 28, 15,  0, 32,
       15, 28, 14, 11, 14, 15, 28, 25,  0, 13, 25, 24,  0, 31, 24,  0, 13,
       25, 24, 30, 15, 24, 19, 14, 25,  0, 30, 11, 24, 30, 25,  0, 14, 15,
        0, 22, 11,  0, 30, 11, 24, 11, 20,  0, 35,  0, 15, 22,  0])


In [ ]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('Torah_data_vocab.csv',index=False)

In [ ]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=300
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'las escrituras de restauracion edicion del nombre verdadero con un contenido tanto de la tanaj y el brit renovado la primera edicion en espanol traducida de la tercera edicion actualizada en ingles sup'
'ervision escritural y administracion doctrinal por el ramyk rabi moshe yosef koniuchowsky editorial sus brazos a yisrae yirmeyahu jeremias 31 31 37 31 los dias vienen dice en que hare brit chadasha bri'
't renovado con bayit yisrael y con bayit yahudah hine yamim baim neum yahweh vecharati et bayit yisrael veet bayit yahudah brit chadasha 32 no sera como el brit que hice con sus padres cuando los tome '
'de la mano y los saque de la tierra de mitzrayim porque ellos por su parte violaron mi brit aunque yo fui un esposo para ellos dice lo chabrit asher karati et avotam beyom hecheziki veyadam lehotsiam m'
'eerets mitzrayim asher hema heferu et brit veanochi baalti vam neum yahweh 33 porque este es el brit que hare con la bayit yisrael despues de esos dias dice yo pondre mi torah en sus 

In [ ]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'las escrituras de restauracion edicion del nombre verdadero con un contenido tanto de la tanaj y el brit renovado la primera edicion en espanol traducida de la tercera edicion actualizada en ingles su'
Target data:  'as escrituras de restauracion edicion del nombre verdadero con un contenido tanto de la tanaj y el brit renovado la primera edicion en espanol traducida de la tercera edicion actualizada en ingles sup'


In [ ]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 128
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((200,), (200,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int64, tf.int64)>


In [ ]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           9472      
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_3 (Dense)             (64, None, 37)            37925     
                                                                 
Total params: 5,294,373
Trainable params: 5,294,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Proyecto/checkpoints/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


In [ ]:
EPOCHS=150
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/150
124/124 [==============================] - ETA: 0s - loss: 2.6892 - accuracy: 0.2065
Epoch 00001: loss improved from inf to 2.68924, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Proyecto/checkpoints/cp_0001.ckpt
124/124 [==============================] - 133s 1s/step - loss: 2.6892 - accuracy: 0.2065
Epoch 2/150
124/124 [==============================] - ETA: 0s - loss: 2.0312 - accuracy: 0.3596
Epoch 00002: loss improved from 2.68924 to 2.03121, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Proyecto/checkpoints/cp_0002.ckpt
124/124 [==============================] - 130s 1s/step - loss: 2.0312 - accuracy: 0.3596
Epoch 3/150
124/124 [==============================] - ETA: 0s - loss: 1.6479 - accuracy: 0.4828
Epoch 00003: loss improved from 2.03121 to 1.64791, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Proyecto/checkpoints/cp_0003.ckpt
124/124 [==============================] - 1

In [ ]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=150, 
                    verbose=1,
                    callbacks=[cp_callback])

In [ ]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)
history_TB=model.fit(dataset, 
                    epochs=100, 
                    verbose=1,
                    callbacks=[tbCallBack])

In [ ]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [ ]:
print(generate_text(model, start_string=u"el rey"))

el rey nuevamente se alegrocon el marques el gato qui una hermosa herencia dijo el rey al marques de carabas misericordia muy ufano con su presa fue al palacio del ogro quiso entrar el gato al oir el ruido del rey yuiente a sucarro y le dijo a su amo sigues mi cosechaban y les dijobuena gente que estan cosechando por el cobro todo el pobre patrimonioel mayor rel ogro habia mandadopreparar para sus amigos que vieramuerto aguardo a que algun conejillo poco cono con la princesa el gato se convirtio en gr


In [ ]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_Torah_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_Torah_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_Torah_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


In [ ]:
!wget https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5?raw=true \
      -O RNN_ElGatoConBotas_model.h5

--2021-11-24 16:36:27--  https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/raw/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5 [following]
--2021-11-24 16:36:28--  https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/raw/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5 [following]
--2021-11-24 16:36:28--  https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5
Resolving raw.githubusercontent.com (raw.github

In [ ]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_Torah_model.h5', 
                           reporthook)

100% | 20 MB | 3.50 MB/s | 5 segundos transcurrido

('RNN_ElGatoConBotas_model.h5', <http.client.HTTPMessage at 0x7fd07d9d35d0>)

In [ ]:
new_model = tf.keras.models.load_model('/content/RNN_Torah_model.h5')

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/data_vocab.csv")
df2.head()